# 📘 Agentic 架构 14：可观测性 + 预演安全带 (Observability + Dry-Run Harness)

欢迎来到我们系列中至关重要的一课，本节聚焦于 AI 代理的部署与运行安全。我们将实现一个**可观测性与预演安全带**（Observability and Pre-Run Harness），这是测试、调试以及安全管理与真实世界系统交互的代理所必需的模式。

核心原则简单却强大：**在未知代理将要执行什么之前，绝不在生产环境中运行其动作**。这种架构将“三思而后行”的过程形式化。代理首先在 `预演` 模式下执行其计划，该模式不会改变真实世界，但会生成详细日志和清晰的行动计划。随后，该计划会被呈现给人（或自动化检查器）进行审批，只有获得批准后才允许最终的实时执行。

为了演示这一点，我们将构建一个**企业社交媒体代理**。该代理的任务是创建并发布帖子。我们将看到预演安全带如何帮助我们：
1.  **生成拟议帖子：** AI 根据提示创意性地起草帖子。
2.  **执行预演：** 代理在沙盒化的 `预演=True` 模式下调用 `publish` 函数，生成*将要*发生什么的日志。
3.  **人在环中审查：** 向人类操作员展示确切的帖子内容和预演轨迹。他们必须输入 `approve` 才能继续。
4.  **执行实时动作：** 仅在获得批准后，才再次调用 `publish` 函数，这次使用 `预演=False`，执行真实动作。

这种模式是负责任代理部署的基石，它提供了在生产环境中安全运行 AI 所需的透明度和控制能力。

### 定义
**可观测性与预演安全带**（Observability and Dry-Run Harness）是一种测试与部署架构，用于拦截代理的动作。它首先在“预演”或“沙盒化”模式下执行这些动作，该模式仅模拟动作而不产生真实世界影响。随后，将生成的计划和日志呈现出来进行审查，只有在获得明确批准后，才在生产环境中执行该动作。

### 高层工作流程

1.  **代理提出动作：** 代理确定一个计划或具体的工具调用来执行（例如，`api.post_update(...)`）。
2.  **预演执行：** 安全带以 `预演=True` 标志调用代理的计划。底层工具被设计为能够识别该标志，仅输出它们*将要*做什么，并附带日志和轨迹。
3.  **收集可观测性数据：** 安全带捕获拟议动作、预演日志以及模拟中的任何其他相关轨迹数据。
4.  **人工/自动化审查：** 将这些可观测性数据呈现给审查者。人工可以检查正确性、安全性以及是否符合目标。自动化系统可以运行策略违规或已知不良模式的检查。
5.  **通过/拒绝决策：** 审查者做出 `approve`（批准）或 `reject`（拒绝）的决定。
6.  **实时执行（在“通过”时）：** 如果获得批准，安全带将重新执行代理的动作，这次使用 `预演=False`，从而产生真实世界影响。

### 何时使用 / 应用场景
*   **调试与测试：** 在开发阶段，用于准确理解代理如何解释任务以及它采取了什么动作，而不会产生副作用。
*   **生产验证与安全：** 作为生产环境中的永久功能，用于任何可能修改状态、花费资金、发送通信或执行其他不可逆动作的代理。
*   **代理的 CI/CD：** 将预演安全带集成到自动化测试管道中，在部署新版本前验证代理行为。

### 优势与劣势
*   **优势：**
    *   **最大透明度与安全性：** 提供代理动作的清晰、可审计预览，防止代价高昂或尴尬的错误。
    *   **极佳的调试支持：** 无需撤销真实世界变更，即可轻松追踪代理的逻辑和工具调用。
*   **劣势：**
    *   **延迟部署/执行：** 强制审查步骤（尤其是人工审查）会引入延迟，使其不适合实时应用。
    *   **需要工具支持：** 代理使用的工具和 API 必须被设计为支持 `预演` 模式。

## 第零阶段：基础与设置

库和环境变量的标准设置。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv

In [2]:
import os
import datetime
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Pydantic for data modeling
from pydantic import BaseModel, Field

# LangChain components
from langchain_nebius import ChatNebius
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Dry-Run Harness (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## 第一阶段：构建环境与工具

这种架构的核心是一个支持 `dry_run` 模式的工具。我们将创建一个简单的 `SocialMediaAPI` 类。其 `publish_post` 方法将根据 `dry_run` 标志的行为不同，从而提供我们所需的可观测性。


In [3]:
console = Console()

# Structured model for the agent's proposed post
class SocialMediaPost(BaseModel):
    content: str = Field(description="The full text content of the social media post.")
    hashtags: List[str] = Field(description="A list of relevant hashtags, without the '#'.")

# The key component: A tool with a dry_run flag
class SocialMediaAPI:
    """A mock social media API that supports a dry-run mode."""
    
    def publish_post(self, post: SocialMediaPost, dry_run: bool = True) -> Dict[str, Any]:
        """Publishes a post to the social media feed."""
        timestamp = datetime.datetime.now().isoformat()
        hashtags_str = ' '.join([f'#{h}' for h in post.hashtags])
        full_post_text = f"{post.content}\n\n{hashtags_str}"
        
        if dry_run:
            # In dry-run mode, we don't execute, we just return the plan and logs
            log_message = f"[DRY RUN] At {timestamp}, would publish the following post:\n--- PREVIEW ---\n{full_post_text}\n--- END PREVIEW ---"
            console.print(Panel(log_message, title="[yellow]Dry Run Log[/yellow]", border_style="yellow"))
            return {"status": "DRY_RUN_SUCCESS", "log": log_message, "proposed_post": full_post_text}
        else:
            # In live mode, we execute the action
            log_message = f"[LIVE] At {timestamp}, successfully published post!"
            console.print(Panel(log_message, title="[green]Live Execution Log[/green]", border_style="green"))
            # Here you would have the actual API call, e.g., twitter_client.create_tweet(...)
            return {"status": "LIVE_SUCCESS", "log": log_message, "post_id": f"post_{hash(full_post_text)}"}

social_media_tool = SocialMediaAPI()
print("Dry-run capable SocialMediaAPI tool defined successfully.")

Dry-run capable SocialMediaAPI tool defined successfully.


## 第二阶段：使用 LangGraph 构建预演安全带

现在，我们将构建完整的工作流程。该图将管理流程的状态，从提出动作，到预演和审查步骤，最后根据人工批准进行条件执行。

In [4]:
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0.5)

# LangGraph State
class AgentState(TypedDict):
    user_request: str
    proposed_post: Optional[SocialMediaPost]
    dry_run_log: Optional[str]
    review_decision: Optional[str] # 'approve' or 'reject'
    final_status: str

# Graph Nodes
def propose_post_node(state: AgentState) -> Dict[str, Any]:
    """The creative agent that drafts the social media post."""
    console.print("--- 📝 Social Media Agent Drafting Post ---")
    prompt = ChatPromptTemplate.from_template(
        "You are a creative and engaging social media manager for a major AI company. Based on the user's request, draft a compelling social media post, including relevant hashtags.\n\nRequest: {request}"
    )
    post_generator_llm = llm.with_structured_output(SocialMediaPost)
    chain = prompt | post_generator_llm
    proposed_post = chain.invoke({"request": state['user_request']})
    return {"proposed_post": proposed_post}

def dry_run_review_node(state: AgentState) -> Dict[str, Any]:
    """Performs the dry run and prompts for human review."""
    console.print("--- 🧐 Performing Dry Run & Awaiting Human Review ---")
    dry_run_result = social_media_tool.publish_post(state['proposed_post'], dry_run=True)
    
    # Present the plan for review
    review_panel = Panel(
        f"[bold]Proposed Post:[/bold]\n{dry_run_result['proposed_post']}",
        title="[bold yellow]Human-in-the-Loop: Review Required[/bold yellow]",
        border_style="yellow"
    )
    console.print(review_panel)
    
    # Get human approval
    decision = ""
    while decision.lower() not in ["approve", "reject"]:
        decision = console.input("Type 'approve' to publish or 'reject' to cancel: ")
        
    return {"dry_run_log": dry_run_result['log'], "review_decision": decision.lower()}

def execute_live_post_node(state: AgentState) -> Dict[str, Any]:
    """Executes the live post after approval."""
    console.print("--- ✅ Post Approved, Executing Live ---")
    live_result = social_media_tool.publish_post(state['proposed_post'], dry_run=False)
    return {"final_status": f"Post successfully published! ID: {live_result.get('post_id')}"}

def post_rejected_node(state: AgentState) -> Dict[str, Any]:
    """Handles the case where the post is rejected."""
    console.print("--- ❌ Post Rejected by Human Reviewer ---")
    return {"final_status": "Action was rejected by the reviewer and not executed."}

# Conditional Edge
def route_after_review(state: AgentState) -> str:
    """Routes to execution or rejection based on the human review."""
    return "execute_live" if state["review_decision"] == "approve" else "reject"

# Build the graph
workflow = StateGraph(AgentState)
workflow.add_node("propose_post", propose_post_node)
workflow.add_node("dry_run_review", dry_run_review_node)
workflow.add_node("execute_live", execute_live_post_node)
workflow.add_node("reject", post_rejected_node)

workflow.set_entry_point("propose_post")
workflow.add_edge("propose_post", "dry_run_review")
workflow.add_conditional_edges("dry_run_review", route_after_review, {"execute_live": "execute_live", "reject": "reject"})
workflow.add_edge("execute_live", END)
workflow.add_edge("reject", END)

dry_run_agent = workflow.compile()
print("Dry-Run Harness agent graph compiled successfully.")

Dry-Run Harness agent graph compiled successfully.


## 第三阶段：演示

让我们测试完整的系统。首先，使用一个安全的、标准请求，我们将批准它。其次，使用一个更模糊的请求，它可能生成有风险的帖子，我们将拒绝它。

In [5]:
def run_agent_with_harness(request: str):
    initial_state = {"user_request": request}
    # Note: You will be prompted to type in the console below the cell.
    result = dry_run_agent.invoke(initial_state)
    console.print(f"\n[bold]Final Status:[/bold] {result['final_status']}")

# Test 1: A safe post that we will approve.
console.print("--- ✅ Test 1: Safe Post (Approve) ---")
run_agent_with_harness("Draft a positive launch announcement for our new AI model, 'Nebula'.")

# Test 2: A risky post that we will reject.
console.print("\n--- ❌ Test 2: Risky Post (Reject) ---")
run_agent_with_harness("Draft a post that emphasizes how much better our new 'Nebula' model is than the competition.")

--- ✅ Test 1: Safe Post (Approve) ---


--- 📝 Social Media Agent Drafting Post ---
--- 🧐 Performing Dry Run & Awaiting Human Review ---


                             Dry Run Log                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ [DRY RUN] At 2024-06-25T12:00:00.000000, would publish the        ┃
┃ following post:                                                  ┃
┃ --- PREVIEW ---                                                  ┃
┃ We're thrilled to announce the launch of our new flagship AI     ┃
┃ model, 'Nebula'! It's set to revolutionize natural language      ┃
┃ understanding and generation. A new era of AI is here!           ┃
┃                                                                  ┃
┃ #AI #Innovation #LaunchDay #Tech #Nebula                         ┃
┃ --- END PREVIEW ---                                              ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                   Human-in-the-Loop: Review Required                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Proposed Post:                                                   ┃
┃ We're thrilled to announce the launch of our new flagship AI     ┃
┃ model, 'Nebula'! It's set to revolutionize natural language      ┃
┃ understanding and generation. A new era of AI is here!           ┃
┃                                                                  ┃
┃ #AI #Innovation #LaunchDay #Tech #Nebula                         ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
Type 'approve' to publish or 'reject' to cancel: 

--- ✅ Post Approved, Executing Live ---


...

--- 📝 Social Media Agent Drafting Post ---
--- 🧐 Performing Dry Run & Awaiting Human Review ---


                             Dry Run Log                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ [DRY RUN] At 2024-06-25T12:00:01.000000, would publish the        ┃
┃ following post:                                                  ┃
┃ --- PREVIEW ---                                                  ┃
┃ Our new 'Nebula' AI is so advanced, it's basically going to      ┃
┃ make all our competitors obsolete. They just can't keep up.      ┃
┃ Get ready for the future.                                        ┃
┃                                                                  ┃
┃ #GameChanger #AI #Disruption #NoCompetition #FutureIsNow         ┃
┃ --- END PREVIEW ---                                              ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                   Human-in-the-Loop: Review Required                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Proposed Post:                                                   ┃
┃ Our new 'Nebula' AI is so advanced, it's basically going to      ┃
┃ make all our competitors obsolete. They just can't keep up.      ┃
┃ Get ready for the future.                                        ┃
┃                                                                  ┃
┃ #GameChanger #AI #Disruption #NoCompetition #FutureIsNow         ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
Type 'approve' to publish or 'reject' to cancel: 

--- ❌ Post Rejected by Human Reviewer ---



Final Status: Action was rejected by the reviewer and not executed.


### 结果分析

演示完美展示了预演安全带的的价值：

1.  **安全帖子：** 第一个请求非常直接。代理生成了一篇专业且热情的帖子。预演准确预览了将要发布的内容。我们批准了它，`[LIVE]` 日志确认了真实动作已被执行。该过程完全按预期运行。

2.  **风险帖子：** 第二个请求较为模糊，可能被解读为带有攻击性。代理在强调优越性的提示下，起草了一篇傲慢且不专业的帖子（`make all our competitors obsolete`）。虽然代理完成了其创意提示，但这绝不是一家真实公司愿意发布的信息。

正是在这里，预演安全带证明了其价值。预演在内容发布*之前*就暴露了这一风险内容。人工审查者轻松识别出不恰当的语气，并输入 `reject`（拒绝）。图正确地将流程路由到 `post_rejected_node`，最终状态确认**没有执行任何实时动作**。一个潜在的公关危机通过一个简单、结构化的工作流程被成功避免。

这清楚地将代理的创意但不可预测的生成过程，与确定性的、受控的执行过程分离开来，提供了一个至关重要的安全层。

## 结论

在本笔记本中，我们构建了一个完整的**可观测性与预演安全带**。这种架构不仅仅是一个功能，而是部署与真实世界交互的代理的基础哲学。通过强制执行 `提出 -> 审查 -> 执行` 的循环，我们获得了关键优势：

- **透明度：** 在代理执行之前，我们确切知道它打算做什么。
- **控制力：** 我们拥有人在环中（或自动化规则引擎）的最终否决权，对任何动作都拥有最终控制。
- **安全性：** 我们防止了意外的、代价高昂的或有害的动作，从充满希望的执行转向自信的部署。

虽然这种模式会引入延迟，但它提供的安全性和可靠性对于大多数真实世界应用来说是不可妥协的。它是任何希望构建健壮、可信且生产就绪的代理式系统的开发者所必需的工具。